<a href="https://colab.research.google.com/github/JaewonLee0217/JW_chatbot/blob/main/JW_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [53]:
with open('intents.json') as file:
  data = json.load(file)
  #데이터 구조는 패턴, 리스폰스, 태그로 구성되어 있고, 

  training_sentences = [] # 실제 어구 
  training_labels = [] #태그 정보들 
  labels = [] # 이 문장이 인사인지 도움인지 뭐를 뜻하는 지 태그 정보 
  responses = [] # 패턴에 대한 반응


#data proprecessing -> 각 리스트에 정보 분배 
for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])


# 라벨 리스트에 태그 정보들 기록 
  if intent['tag'] not in labels:
    labels.append(intent['tag'])

#라벨 리스트에 담은 길이가 클래스의 총 개수로 정해준다.
num_classes = len(labels)

In [54]:
#인코딩 실행 sklearn의 전처리 라이브러리 불러오서 각 패턴에 대한 태그들,
# 뭐라뭐라 말했을 떄 그 말에 대한 단어들을 모아둔 training_labels 에서 인코더를 fit 시키고 
# 그것을 가주고 숫자 형태로 transforming 시켜 준다. 
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [47]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [55]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

#tokenizer는 텐서플로우의 keras라이브러리 꺼 사용. 
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
#여기서 문장에서 각 토큰 들 짤라주고 
tokenizer.fit_on_texts(training_sentences)

#짜른 토큰들을 word to index화 시킨다. 
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)

# 그럼 sequence에는 각 단어 문장의 단어마다 숫자로 표현되어 있지만 길이가 
# 서로 다르므로 제로 패딩을 통해서 각 문장마다 길이 동일하게 만들어주고 트레이닝을 진행해야 한다
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)


In [59]:
#모델 훈련
#모델을 먼저 텐서플로우 케라스 모델에서 세퀀셜가져오고


#keras.layer 라이브러리에서 임베딩함수로 인풋이 들어갈 부분을 만들어준다. 
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [60]:
epochs = 490
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/490
2/2 [==============================] - 0s 12ms/step - loss: 2.0787 - accuracy: 0.1225
Epoch 2/490
2/2 [==============================] - 0s 14ms/step - loss: 2.0776 - accuracy: 0.1225
Epoch 3/490
2/2 [==============================] - 0s 12ms/step - loss: 2.0768 - accuracy: 0.2449
Epoch 4/490
2/2 [==============================] - 0s 13ms/step - loss: 2.0762 - accuracy: 0.2756
Epoch 5/490
2/2 [==============================] - 0s 13ms/step - loss: 2.0755 - accuracy: 0.3368
Epoch 6/490
2/2 [==============================] - 0s 12ms/step - loss: 2.0750 - accuracy: 0.2756
Epoch 7/490
2/2 [==============================] - 0s 11ms/step - loss: 2.0744 - accuracy: 0.2449
Epoch 8/490
2/2 [==============================] - 0s 11ms/step - loss: 2.0739 - accuracy: 0.2143
Epoch 9/490
2/2 [==============================] - 0s 9ms/step - loss: 2.0734 - accuracy: 0.2652
Epoch 10/490
2/2 [==============================] - 0s 9ms/step - loss: 2.0723 - accuracy: 0.3368
Epoch 11/490
2/2 [===

최종 loss: 0.6715 - accuracy: 0.9388


In [61]:
# to save the trained model
model.save("JW_model_2")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: JW_model_2/assets


In [ ]:
pip install colorama


In [62]:

import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        #print('result : ',result)
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        #print('tag : ',tag)

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Hi
result :  [[0.03010814 0.01078339 0.03460111 0.08598869 0.6074504  0.00819192
  0.0414453  0.18143098]]
tag :  ['greeting']
ChatBot: Hi there
User: Who are you?
result :  [[0.2224784  0.00356181 0.00604405 0.17680888 0.2193965  0.00768735
  0.31941083 0.04461214]]
tag :  ['name']
ChatBot: You can call me Joana.
User: What are you?
result :  [[0.21921374 0.00356596 0.00594932 0.17610598 0.21728979 0.00751528
  0.32617855 0.04418132]]
tag :  ['name']
ChatBot: I'm Joana!
User: 

KeyboardInterrupt: ignored

In [ ]:
for i in data['intents']:
  print(i['tag'])

In [52]:
labels[5]


'help'